###### 2020-11-12 목요일

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

import time
import pandas as pd
import numpy as np


## 1. Selenium으로 인스타홈페이지 접속

In [22]:
path='./chrome_Driver/chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get("http://instagram.com")

time.sleep(5)

## 2. Selenium으로 인스타아이디 로그인

In [23]:
## 로그인
id_input_temp, pwd_input_temp = tuple(driver.find_elements_by_css_selector('div.gr27e > div.EPjEi > form.HmktE > div > div.-MzZI'))

id_input  = id_input_temp.find_element_by_css_selector('div > label')
pwd_input = pwd_input_temp.find_element_by_css_selector('div > label')

id_input.send_keys('rnrmfduqns1@gmail.com')   # 당신의 인스타아이디를 입력하세요
pwd_input.send_keys('zmfhfflddyddlqslek')       # 당신의 인스타비밀번호를 입력하세요

pwd_input.submit()

time.sleep(5)

## 3. 크롤링 하고자 하는 유명인의 아이디 찾고 홈페이지 띄우기

In [24]:
# 셀럽아이디입력

find_id = driver.find_element_by_css_selector('nav > div._8MQSO > div > div > div.LWmhU > input')
find_id.send_keys('#고영욱')  # 크롤링 하고자하는 유명인의 "인스타 아이디"를 입력해주세요
time.sleep(5)

# 셀럽아이디입력하고 홈페이지 띠우기
for_click = driver.find_element_by_css_selector('nav > div._8MQSO > div > div > div.LWmhU > div.drKGC > div > a')
for_click.click()
time.sleep(5)

## 4. 게시물 크롤링

In [31]:
insta_list = driver.find_element_by_css_selector('body > #react-root > section._9eogI > main.SCxLW > article.KC1QD > div.EZdmt > div > div > div.Nnq7C')
big_list = insta_list.find_elements_by_css_selector('div.Nnq7C > div.v1Nh3')

comment_list = []

for small_list in tqdm_notebook(big_list):
    for_click_list = small_list.find_elements_by_css_selector('div.v1Nh3 > a')
    
    for element in tqdm_notebook(for_click_list):
        
        # 개시물 오픈
        element.send_keys('\n')
        time.sleep(5)
        
        # 댓글란 최대한 열기
        while 1:
            try:
                expansion_comment = driver.find_element_by_css_selector('button.dCJp8')
                expansion_comment.send_keys('\n')
                time.sleep(3)
            except :
                print('댓글 페이지를 최대로 확장하였습니다')
                break
        
        
        # 게시글 크롤링
        comment_body_list = driver.find_elements_by_css_selector('body > div._2dDPU > div.zZYga > div.PdwC2 > article > div.eo2As > div.EtaWk > ul.XQXOT > ul.Mr508')
        n = len(comment_body_list)
        
        for idx in range(n):
            comment = comment_body_list[idx].find_element_by_css_selector('div > li > div > div.C7I1f > div.C4VMK > span')
            comment_list.append(comment.text)
        print('현재까지 긁어온 댓글의 수 :', len(comment_list))
        
        
        # 게시물 닫기
        close_bnt = driver.find_element_by_css_selector('body > div._2dDPU > div.Igw0E > button')
        close_bnt.send_keys('\n')
        
    time.sleep(5)

        

댓글 페이지를 최대로 확장하였습니다
현재까지 긁어온 댓글의 수 : 68



댓글 페이지를 최대로 확장하였습니다
현재까지 긁어온 댓글의 수 : 74



댓글 페이지를 최대로 확장하였습니다
현재까지 긁어온 댓글의 수 : 652




## 5. 크롤링한 댓글 데이터프레임화 하고 csv파일로 만들기

In [32]:
data = pd.DataFrame({'댓글' : comment_list})
    
data.head()

,댓글
0,신고하고왔어요...
1,무슨소통을 하실려고
2,고영욱 sns하고 조두순 출소하고 손정우 구속영장 기각되고.....이게 2020년이...
3,소통은 오프라인으로 지인들이랑 만나서 해...
4,스토리 공유하겠습니다...어디서 스멀스멀 기어나올려고...🤔


In [33]:
data.to_csv('./크롤링한 댓글 데이터/고영욱.csv', encoding='utf-8-sig')